In [1]:
import sys
from pathlib import Path
wdir = Path.cwd().parent.absolute()
sys.path.insert(1, str(wdir))

from models.EEGModels import EEGNet, ShallowConvNet, DeepConvNet, EEGNet_ChanRed, CapsEEGNet, TCNet
import torch
import os

In [2]:
model = EEGNet(nb_classes = 2, Chans = 14, Samples = 128, dropoutRate = 0.1, kernLength = 16, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = 0
for name, param in model.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

Layer: block1.0.weight, Parameters: 1024
Layer: block1.1.weight, Parameters: 64
Layer: block1.1.bias, Parameters: 64
Layer: block1.2.weight, Parameters: 7168
Layer: block1.3.weight, Parameters: 512
Layer: block1.3.bias, Parameters: 512
Layer: block2.0.depthwise.weight, Parameters: 8192
Layer: block2.0.pointwise.weight, Parameters: 32768
Layer: block2.1.weight, Parameters: 64
Layer: block2.1.bias, Parameters: 64
Layer: dense.weight, Parameters: 512
Layer: dense.bias, Parameters: 2
Total Parameters: 50946


In [3]:
model_SEED = EEGNet_ChanRed(nb_classes = 3, Chans = 62, InnerChans=14, Samples = 200, dropoutRate = 0.1, kernLength = 25, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = 0
for name, param in model_SEED.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

Layer: chan_reduction.weight, Parameters: 868
Layer: chan_reduction.bias, Parameters: 14
Layer: block1.0.weight, Parameters: 1600
Layer: block1.1.weight, Parameters: 64
Layer: block1.1.bias, Parameters: 64
Layer: block1.2.weight, Parameters: 7168
Layer: block1.3.weight, Parameters: 512
Layer: block1.3.bias, Parameters: 512
Layer: block2.0.depthwise.weight, Parameters: 8192
Layer: block2.0.pointwise.weight, Parameters: 32768
Layer: block2.1.weight, Parameters: 64
Layer: block2.1.bias, Parameters: 64
Layer: dense.weight, Parameters: 1152
Layer: dense.bias, Parameters: 3
Total Parameters: 53045


In [4]:
x = torch.randn(8, 1, 62, 200)
y = model_SEED(x)
print(y.size())

torch.Size([8, 3])


/Users/juliengadonneix/anaconda3/envs/intern3A/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1712608633180/work/aten/src/ATen/native/Convolution.cpp:1032.)
  return F.conv2d(input, weight, bias, self.stride,


In [5]:
model_capseegnet = CapsEEGNet(nb_classes=2)

total_params = 0
for name, param in model_capseegnet.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

Layer: block_1_2.0.weight, Parameters: 512
Layer: block_1_2.1.weight, Parameters: 8
Layer: block_1_2.1.bias, Parameters: 8
Layer: block_1_2.3.weight, Parameters: 224
Layer: block_1_2.4.weight, Parameters: 16
Layer: block_1_2.4.bias, Parameters: 16
Layer: primaryCaps.capsules.0.weight, Parameters: 4608
Layer: primaryCaps.capsules.0.bias, Parameters: 32
Layer: primaryCaps.capsules.1.weight, Parameters: 4608
Layer: primaryCaps.capsules.1.bias, Parameters: 32
Layer: primaryCaps.capsules.2.weight, Parameters: 4608
Layer: primaryCaps.capsules.2.bias, Parameters: 32
Layer: primaryCaps.capsules.3.weight, Parameters: 4608
Layer: primaryCaps.capsules.3.bias, Parameters: 32
Layer: primaryCaps.capsules.4.weight, Parameters: 4608
Layer: primaryCaps.capsules.4.bias, Parameters: 32
Layer: primaryCaps.capsules.5.weight, Parameters: 4608
Layer: primaryCaps.capsules.5.bias, Parameters: 32
Layer: primaryCaps.capsules.6.weight, Parameters: 4608
Layer: primaryCaps.capsules.6.bias, Parameters: 32
Layer: pri

In [6]:
x = torch.randn(8, 1, 14, 128)
y = model(x)
print(y.size())

torch.Size([8, 2])


In [7]:
model_tcnet = TCNet(nb_classes=2)


total_params = 0
for name, param in model_tcnet.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

/Users/juliengadonneix/anaconda3/envs/intern3A/lib/python3.9/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Layer: PatchPartition.weight, Parameters: 5376
Layer: PatchPartition.bias, Parameters: 32
Layer: primaryCaps.capsules.0.weight, Parameters: 576
Layer: primaryCaps.capsules.0.bias, Parameters: 8
Layer: primaryCaps.capsules.1.weight, Parameters: 576
Layer: primaryCaps.capsules.1.bias, Parameters: 8
Layer: primaryCaps.capsules.2.weight, Parameters: 576
Layer: primaryCaps.capsules.2.bias, Parameters: 8
Layer: primaryCaps.capsules.3.weight, Parameters: 576
Layer: primaryCaps.capsules.3.bias, Parameters: 8
Layer: primaryCaps.capsules.4.weight, Parameters: 576
Layer: primaryCaps.capsules.4.bias, Parameters: 8
Layer: primaryCaps.capsules.5.weight, Parameters: 576
Layer: primaryCaps.capsules.5.bias, Parameters: 8
Layer: primaryCaps.capsules.6.weight, Parameters: 576
Layer: primaryCaps.capsules.6.bias, Parameters: 8
Layer: primaryCaps.capsules.7.weight, Parameters: 576
Layer: primaryCaps.capsules.7.bias, Parameters: 8
Layer: primaryCaps.capsules.8.weight, Parameters: 576
Layer: primaryCaps.capsu

In [8]:
x = torch.randn(8, 14, 48, 128)
out = model_tcnet(x)
print(out.shape)

torch.Size([8, 2])


In [10]:
print(os.cpu_count())
print(torch.cuda.device_count()) 
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(device)
print(device.type == 'cuda')
if device.type == 'cuda':
    properties = torch.cuda.get_device_properties(device)
    print(properties)

8
0
mps
False


In [11]:
import subprocess
import plistlib

def get_gpu_info():
    """Get information about the GPU on macOS."""
    # Run system_profiler command and capture the output
    result = subprocess.run(['system_profiler', '-xml', 'SPDisplaysDataType'], capture_output=True)
    if result.returncode == 0:
        # Parse the XML output
        display_info = plistlib.loads(result.stdout)
        # Extract GPU information
        gpu_count = len(display_info[0]['_items'])
        return gpu_count
    else:
        print("Error:", result.stderr)
        return None

gpu_count = get_gpu_info()
if gpu_count is not None:
    print("Number of GPUs:", gpu_count)


Number of GPUs: 1


In [12]:
import time
import torch

start_time = time.time()

# syncrocnize time with cpu, otherwise only time for oflaoding data to gpu would be measured
torch.mps.synchronize()

a = torch.ones(4000,4000, device="mps")
for _ in range(200):
   a +=a

elapsed_time = time.time() - start_time
print( "GPU Time: ", elapsed_time)

GPU Time:  0.28950977325439453


In [13]:
t = torch.randn(8,8)
print(t)
torch.reshape(t, (-1, 2, 8))

tensor([[-0.8986,  1.4910, -0.5834,  0.4678, -1.1204, -0.8281, -0.8942,  0.3483],
        [ 0.0297, -0.1527,  1.0798,  0.5965, -0.1218, -0.5234,  0.9439,  0.1153],
        [ 0.2668,  1.4996, -0.1968, -0.8245,  0.6688, -1.0775, -1.0743, -0.1578],
        [ 0.9854,  0.0943, -1.3081,  1.4217, -1.2786, -0.3437,  0.6738,  0.2421],
        [ 0.9156, -0.1657, -1.1023,  1.7003, -0.3087,  0.6996,  0.5284, -0.1102],
        [ 0.4162, -0.6433, -1.1270,  0.9608,  0.4103, -1.5273,  0.5295,  1.3108],
        [-0.6538, -0.3917, -0.2714,  0.8145,  0.3435,  1.7909,  0.9297,  0.6730],
        [ 0.6595, -0.2418, -0.0284, -0.2539, -2.6989, -0.0140, -0.8288, -2.1690]])


tensor([[[-0.8986,  1.4910, -0.5834,  0.4678, -1.1204, -0.8281, -0.8942,
           0.3483],
         [ 0.0297, -0.1527,  1.0798,  0.5965, -0.1218, -0.5234,  0.9439,
           0.1153]],

        [[ 0.2668,  1.4996, -0.1968, -0.8245,  0.6688, -1.0775, -1.0743,
          -0.1578],
         [ 0.9854,  0.0943, -1.3081,  1.4217, -1.2786, -0.3437,  0.6738,
           0.2421]],

        [[ 0.9156, -0.1657, -1.1023,  1.7003, -0.3087,  0.6996,  0.5284,
          -0.1102],
         [ 0.4162, -0.6433, -1.1270,  0.9608,  0.4103, -1.5273,  0.5295,
           1.3108]],

        [[-0.6538, -0.3917, -0.2714,  0.8145,  0.3435,  1.7909,  0.9297,
           0.6730],
         [ 0.6595, -0.2418, -0.0284, -0.2539, -2.6989, -0.0140, -0.8288,
          -2.1690]]])